# Goal

Save nii

# Imports

In [1]:
# NYU
code_src    = "/gpfs/home/gologr01"
data_src    = "/gpfs/data/oermannlab/private_data/DeepPit"

In [2]:
# UMich 
# code src: "/home/labcomputer/Desktop/Rachel"
# data src: "../../../../..//media/labcomputer/e33f6fe0-5ede-4be4-b1f2-5168b7903c7a/home/rachel/"

In [3]:
import os

# Paths to (1) code (2) data (3) saved models (4) saved metadata
deepPit_src = f"{code_src}/DeepPit"
obelisk_src = f"{code_src}/OBELISK"

# saved models, dset metadata
model_src  = f"{data_src}/saved_models"
dsetmd_src = f"{data_src}/saved_dset_metadata"

# dsets
dsets_src    = f"{data_src}/PitMRdata"

# key,val = dset_name, path to top level dir
dset_dict = {
    "ABIDE"                  : f"{dsets_src}/ABIDE",
    "ABVIB"                  : f"{dsets_src}/ABVIB/ABVIB",
    "ADNI1_Complete_1Yr_1.5T": f"{dsets_src}/ADNI/ADNI1_Complete_1Yr_1.5T/ADNI",
    "AIBL"                   : f"{dsets_src}/AIBL/AIBL",
    "ICMB"                   : f"{dsets_src}/ICMB/ICBM",
    "PPMI"                   : f"{dsets_src}/PPMI/PPMI",
}

# print
print("Folders in dset src: ", end=""); print(*os.listdir(dsets_src), sep=", ")

Folders in dset src: ICMB, ABVIB (1).zip, central.xnat.org, ADNI, PPMI, Oasis_long, samir_labels, ACRIN-FMISO-Brain, LGG-1p19qDeletion, REMBRANDT, AIBL, CPTAC-GBM, TCGA-GBM, TCGA-LGG, ABVIB, ABIDE, AIBL.zip


In [4]:
# imports

# Utilities
import os
import sys
import time
import glob
import pickle
from pathlib import Path
# sys.path.append('/gpfs/home/gologr01/DeepPit')
# sys.path.append('/gpfs/home/gologr01/OBELISK')

# Numpy torch pandas
import torch

# imports
import SimpleITK as sitk
import meshio
from helpers.preprocess import seg2mask, get_data_dict

# MR data

In [5]:
dset_name = "ABVIB" #"AIBL" #"ABVIB" #"ICMB" # "AIBL" # "ABVIB" #PPMI
dset_src  = dset_dict[dset_name]

# load files
with open(f"{dsetmd_src}/{dset_name}_dset_metadata.pkl", "rb") as f:
    q_t1 = pickle.load(f)

with open(f"{dsetmd_src}/{dset_name}_fnames.txt", "rb") as f:
    fnames = pickle.load(f)

In [6]:
print(dset_name, len(fnames)) #, *fnames, sep="\n")

ABVIB 403


# Test/Cleanup

In [9]:
corrected = []
uncorrected = []
multiple    = []

def is_corrected(f):
    nii_paths     = glob.glob(f"{f}/*nii.nii")
    #cor_nii_paths = glob.glob(f"{f}/*corrected_n4.nii")
    #las_nii_paths = glob.glob(f"{f}/*las_corrected_n4.nii")
    
    #nii_paths = [p for p in nii_paths if p not in cor_nii_paths]
    
    if len(nii_paths) == 1: 
        corrected.append(f)
        return True
    
    if len(nii_paths) == 0: 
        uncorrected.append(f)
        return False
    
    if len(nii_paths) > 1: 
        multiple.append(f)
        return True  
                
for f in fnames:
    is_corrected(f)
    
print(f"Corrected: {len(corrected)}, TODO: {len(uncorrected)}, Dupl: {len(multiple)}")

Corrected: 0, TODO: 403, Dupl: 0


In [10]:
#print("Uncorrected: ", *uncorrected, sep="\n")

#os.listdir(uncorrected[0])
#uncorrected_nii = [os.path.join(f, os.listdir(f)[0]) for f in uncorrected]
#uncorrected_nii

# Very strange
# os.remove('/gpfs/data/oermannlab/private_data/DeepPit/PitMRdata/ABIDE/ABIDE/50455/MP-RAGE/2000-01-01_00_00_00.0/S165455/._ABIDE_50455_MRI_MP-RAGE_br_raw_20120831000745302_S165455_I329465.nii')

# # delete multiple
# for f in multiple:
#     nii_paths = glob.glob(f"{f}/*corrected_n4_corrected_n4.nii")
#     for p in nii_paths:
#         os.remove(p)

# process uncorrected
# from FAIMED3D 02_preprocessing
# and https://simpleitk.readthedocs.io/en/master/link_N4BiasFieldCorrection_docs.html

# print("tot ", len(uncorrected))

# uncorrected_nii = [os.path.join(f, os.listdir(f)[0]) for f in uncorrected]

# count = 0
# for mr_path in uncorrected_nii: 
#     # print
#     print(count, mr_path, flush=True)
#     count += 1
    
#     # Read in image
#     inputImage = sitk.ReadImage(mr_path, sitk.sitkFloat32)
    
#     # Mask the head to estimate bias
#     maskImage = sitk.OtsuThreshold(inputImage, 0, 1, 200)
    
#     # Set corrector
#     corrector = sitk.N4BiasFieldCorrectionImageFilter()
#     corrector.SetMaximumNumberOfIterations([3] * 3)
#     corrected_image = corrector.Execute(inputImage, maskImage)

#     # write image
#     corrected_fn = mr_path[:-4] + "_corrected_n4.nii"
#     sitk.WriteImage(corrected_image, corrected_fn)

# Get chunk

In [11]:
import os

try:
    taskid = int(os.getenv('SLURM_ARRAY_TASK_ID'))
except:
    taskid = 0
    
n_total = len(uncorrected)

chunk_len = 5    
chunks    = [range(i,min(i+chunk_len, n_total)) for i in range(0, n_total, chunk_len)]

print(f"N_chunks = {len(chunks)}")
# print(f"Array Task ID: {taskid}")
# print(f"Array ID: {os.getenv('SLURM_ARRAY_TASK_ID')}")
# print(f"Job ID: {os.getenv('SLURM_JOB_ID')}")
#print(*chunks, sep="\n")

task_chunk = chunks[taskid]

N_chunks = 81


In [12]:
def read_dcm(fn):
    im = sitk.ReadImage(sitk.ImageSeriesReader_GetGDCMSeriesFileNames(fn), sitk.sitkFloat32)
    return im

def read_nii(fn):
    im = sitk.ReadImage(im_path, sitk.sitkFloat32)    
    return im

# dcm
    #reader = sitk.ImageSeriesReader()
    #dicom_names = reader.GetGDCMSeriesFileNames(fn)
    #reader.SetFileNames(dicom_names)
    #im = reader.Execute() 
    

# Process

In [13]:
uncorrected_chunk = [uncorrected[i] for i in task_chunk]
print(len(uncorrected_chunk), *uncorrected_chunk, sep="\n")

5
/gpfs/data/oermannlab/private_data/DeepPit/PitMRdata/ABVIB/ABVIB/3830/MPRAGE/2012-06-18_11_46_49.0/S341930
/gpfs/data/oermannlab/private_data/DeepPit/PitMRdata/ABVIB/ABVIB/7036/T1_SPGR/2011-08-10_12_32_48.0/S354724
/gpfs/data/oermannlab/private_data/DeepPit/PitMRdata/ABVIB/ABVIB/263/MPRAGE/2011-11-15_11_26_09.0/S343566
/gpfs/data/oermannlab/private_data/DeepPit/PitMRdata/ABVIB/ABVIB/263/MPRAGE_S2_DIS3D/2011-11-15_11_26_09.0/S343575
/gpfs/data/oermannlab/private_data/DeepPit/PitMRdata/ABVIB/ABVIB/60008/MPRAGE/2012-03-21_09_59_38.0/S342266


In [14]:
# from FAIMED3D 02_preprocessing
# and https://simpleitk.readthedocs.io/en/master/link_N4BiasFieldCorrection_docs.html

count = 0
for mr_path in uncorrected_chunk: 
    
    #start = time.time()
    
    # print
    print(count, mr_path, flush=True)
    count += 1
    
    # Read in image
    try:
        inputImage = read_nii(mr_path)
    except:
        inputImage = read_dcm(mr_path) 

    # write image as nii
    corrected_fn = f"{mr_path}/nii.nii"
    sitk.WriteImage(inputImage, corrected_fn)
    
    #elapsed = time.time() - start
    #print(f"Elapsed: {elapsed:0.2f} s")

0 /gpfs/data/oermannlab/private_data/DeepPit/PitMRdata/ABVIB/ABVIB/3830/MPRAGE/2012-06-18_11_46_49.0/S341930
1 /gpfs/data/oermannlab/private_data/DeepPit/PitMRdata/ABVIB/ABVIB/7036/T1_SPGR/2011-08-10_12_32_48.0/S354724
2 /gpfs/data/oermannlab/private_data/DeepPit/PitMRdata/ABVIB/ABVIB/263/MPRAGE/2011-11-15_11_26_09.0/S343566
3 /gpfs/data/oermannlab/private_data/DeepPit/PitMRdata/ABVIB/ABVIB/263/MPRAGE_S2_DIS3D/2011-11-15_11_26_09.0/S343575
4 /gpfs/data/oermannlab/private_data/DeepPit/PitMRdata/ABVIB/ABVIB/60008/MPRAGE/2012-03-21_09_59_38.0/S342266


In [ ]:
print("Done.")